# Feature Selection and Hyperparameter Tuning

### Project Goal  : build a model predicting  `pa_approved` based on the following features<br>
- `bin`  The BIN of the payer(insurance company) for the claim.
- `drug`   The drug that was associated with the claim.
- `reject_code`   If the claim was rejected, what was the associated rejection code. It tells you the reason why the claim has been rejected.
- `correct_diagnosis`    Flag for information provided by the provider indicating that the patient has the correct diagnosis for the associated drug.
- `tried_and_failed`    Flag for information provided by the provider indicating that the patient has tried and failed the relevant generic alternatives.
- `contraindication`    Flag for information provided by the provider indicating that the patient has an associated contraindication for the medication requested.

### Summary of what have been done in the previous notebooks
- We concluded that the feature `reject_code` is redundant and dropped from our model.
- We noticed that there were some nontrivial interaction between features and added the interaction terms of the pairs(`bin`, `drug`), (`tried_and_faild`,`drug`),(`contraindication`,`drug`). 

### Summary of this notebook
We will select features and tune hyperparameters via cross validation on preprocessing data set with respect to each key performance indicator. More precisely,  we will choose features and hyperparameters that optimize `accuracy_score`,`precision`, `true_positive_rate`, `false_positive_rate`, `f1` and `roc_auc` respectively. Note that this feature selection is different from the feature selection that have been done in EDA notebook.<br><br>
 &emsp; - In EDA we did not take into account the target variable when performing feature selection.<br>
&emsp; - In this note we perform feature selection using the target variable.<br><br>
In the end, we will be using `roc_auc` as our key performance indicator but we will also build machine learning models based on other key performance indicators i.e. `accuracy_score`,`precision`, `true_positive_rate`, `false_positive_rate` and `f1`.

As a result, we will choose logistic regression as our machine learning model.

## 0. Load Libraries and Data

Recall that we split data into three parts 'pre'(`X_pre`,`y_pre`), 'train'(`X_train`, `y_train`) and 'test'(`X_test`, `y_test`).  `X_pre_inter`,`X_train_inter` and`X_test_inter` are `X_pre`,`X_train` and `X_test` with columns encoding interaction terms added.<br>
- `X_pre(_inter)`,`y_pre(_inter)` will be used for the exploratory data analysis, feature selection and hyperparameter tuning.
- `X_train`, `y_train` will be used for model selection.
- `X_test`, `y_test` will be used for model assessment.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score,recall_score, f1_score, roc_auc_score, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data
X_pre = pd.read_csv('data/X_pre',index_col = 0)
X_train = pd.read_csv('data/X_train',index_col = 0)
X_inter_pre = pd.read_csv('data/X_inter_pre',index_col = 0)
X_inter_train = pd.read_csv('data/X_inter_train',index_col = 0)
y_pre = pd.read_csv('data/y_pre',index_col = 0)
y_train = pd.read_csv('data/y_train',index_col = 0)

## 1. Feature Selection and  Hyperparameter Tuning

Feature selection and hyperparameter tuning scheme will differ from models to models. 
For example, when selecting features for logistic regression, I wil use built in reuglarization method.
Whereas, for support vector machine, I will use forward stepwise feature selection.
I will explain this in detail right before I do feature selection and hyperparameter tuning.

- `paras` : a short for hyperparameters. This is where we will store tuned hyperparameters.<br>
- `feats` : a short for featrues. This is where we will store selected features.<br>
- `kfold` : a kfold object for 5-fold cross validation.

In [3]:
# tuned hyperparameters
paras = pd.DataFrame(data = {'accuracy_score' : [{},{},{},{},{},{},{}],
                             'precision' : [{},{},{},{},{},{},{}],
                             'true_positive_rate' : [{},{},{},{},{},{},{}],
                             'false_positive_rate' : [{},{},{},{},{},{},{}],
                             'f1' : [{},{},{},{},{},{},{}],
                             'roc_auc' : [{},{},{},{},{},{},{}]},
                     index=['Base','Log', 'LDA','SVM','DT','RF','XGB'])
# selected features
feats = pd.DataFrame(data = {'accuracy_score' : [[],[]],
                             'precision' : [[],[]],
                             'true_positive_rate' : [[],[]],
                             'false_positive_rate' : [[],[]],
                             'f1' : [[],[]],
                             'roc_auc' : [[],[]]},
                     index=['LDA','SVM'])
# kfold object
kfold = StratifiedKFold(n_splits = 5,
                        shuffle = True,
                        random_state = 623)

In [4]:
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,{},{},{},{},{},{}
LDA,{},{},{},{},{},{}
SVM,{},{},{},{},{},{}
DT,{},{},{},{},{},{}
RF,{},{},{},{},{},{}
XGB,{},{},{},{},{},{}


In [5]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,[],[],[],[],[],[]
SVM,[],[],[],[],[],[]


### - Baseline Model-

- Our baseline model identically predicts all prior authorizaitons to be approved. 
- There is no hyperparameter to tune or features to select for this model.

### -Logistic Regression-

- For feature selection, we use built in regularization method. 
- we will tune regularization constant `C` and the `penalty` function via cross validation.
- Note that we are using interaction terms of the pairs (`bin`, `drug`), (`tried_and_faild`,`drug`),(`contraindication`,`drug`).
- we will be using `X_pre` and `y_pre` for the cross validation.

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
# hyperparameters
Cs = [.01, .1, 1, 10, 100]
penalties = ['none', 'l1','l2','elasticnet']
# cross validation key performance indicators
acses = np.zeros((5, len(Cs),len(penalties)))
pres = np.zeros((5, len(Cs),len(penalties)))
tprs = np.zeros((5, len(Cs),len(penalties)))
fprs = np.zeros((5, len(Cs),len(penalties)))
f1s = np.zeros((5, len(Cs),len(penalties)))
aucs = np.zeros((5, len(Cs),len(penalties)))

In [8]:
# hyperparameter tuning via cross validation
i=0
for train_index, test_index in kfold.split(X_inter_pre, y_pre) :
    j=0
    # loop through various regularization constants
    for C in Cs :
        k=0
        # loop through various penalty functions
        for penalty in penalties :
            X_tt = X_inter_pre.iloc[train_index, :].copy()
            X_ho = X_inter_pre.iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()
            # we have to use different solver for different penalty function
            if penalty == 'l1' :
                log_reg = LogisticRegression(penalty = penalty, C=C, solver = 'liblinear', max_iter = 100)
            elif penalty == 'elasticnet' :
                log_reg = LogisticRegression(penalty = penalty, C=C, solver = 'saga', max_iter = 100, l1_ratio = .5)
            else:
                log_reg = LogisticRegression(penalty = penalty, C=C, max_iter = 100)
                
            log_reg.fit(X_tt.values, y_tt.values)
            pred = log_reg.predict(X_ho.values)
            pred_proba = log_reg.predict_proba(X_ho.values)[:, 1]
            
            acses[i,j,k] = accuracy_score(y_ho.values, pred)
            conf_mat = confusion_matrix(y_ho, pred)
            pres[i,j,k] = precision_score(y_ho, pred)
            tprs[i,j,k] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
            fprs[i,j,k] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
            f1s[i,j,k] = f1_score(y_ho, pred)
            aucs[i,j,k] = roc_auc_score(y_ho, pred_proba)
            k=k+1
        j=j+1      
    i=i+1

In [9]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
cv_max = np.max(np.max(cv, axis = 0),axis = 0)
penalty_index = np.argwhere(cv == cv_max)[0][1]
C_index = np.argwhere(cv == cv_max)[0][0]
paras.loc['Log', 'accuracy_score']['penalty'] = penalties[penalty_index]
paras.loc['Log', 'accuracy_score']['C'] = Cs[C_index]

# with respect to the precision
cv = np.mean(aucs, axis = 0)
cv_max = np.max(np.max(cv, axis = 0),axis = 0)
penalty_index = np.argwhere(cv == cv_max)[0][1]
C_index = np.argwhere(cv == cv_max)[0][0]
paras.loc['Log', 'precision']['penalty'] =  penalties[penalty_index]
paras.loc['Log', 'precision']['C'] = Cs[C_index]

# with respect to the true positive rate
cv = np.mean(aucs, axis = 0)
cv_max = np.max(np.max(cv, axis = 0),axis = 0)
penalty_index = np.argwhere(cv == cv_max)[0][1]
C_index = np.argwhere(cv == cv_max)[0][0]
paras.loc['Log', 'true_positive_rate']['penalty'] =  penalties[penalty_index]
paras.loc['Log', 'true_positive_rate']['C'] = Cs[C_index]

# with respect to the false positive rate
cv = np.mean(aucs, axis = 0)
cv_max = np.max(np.max(cv, axis = 0),axis = 0)
penalty_index = np.argwhere(cv == cv_max)[0][1]
C_index = np.argwhere(cv == cv_max)[0][0]
paras.loc['Log', 'false_positive_rate']['penalty'] =  penalties[penalty_index]
paras.loc['Log', 'false_positive_rate']['C'] = Cs[C_index]

# with respect to the f1_score
cv = np.mean(aucs, axis = 0)
cv_max = np.max(np.max(cv, axis = 0),axis = 0)
penalty_index = np.argwhere(cv == cv_max)[0][1]
C_index = np.argwhere(cv == cv_max)[0][0]
paras.loc['Log', 'f1']['penalty'] =  penalties[penalty_index]
paras.loc['Log', 'f1']['C'] = Cs[C_index]

# with respect to the roc_auc_score
cv = np.mean(aucs, axis = 0)
cv_max = np.max(np.max(cv, axis = 0),axis = 0)
penalty_index = np.argwhere(cv == cv_max)[0][1]
C_index = np.argwhere(cv == cv_max)[0][0]
paras.loc['Log', 'roc_auc']['penalty'] =  penalties[penalty_index]
paras.loc['Log', 'roc_auc']['C'] = Cs[C_index]

In [10]:
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,"{'penalty': 'l2', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}"
LDA,{},{},{},{},{},{}
SVM,{},{},{},{},{},{}
DT,{},{},{},{},{},{}
RF,{},{},{},{},{},{}
XGB,{},{},{},{},{},{}


In [11]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,[],[],[],[],[],[]
SVM,[],[],[],[],[],[]


### -Linear Discriminant Analysis-

- For feature selection, we will try every possible combination of features.
- There is no hyperparameter to tune for this model.
- we will be using `X_pre` and `y_pre` for hyperparameter tuning.

In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [13]:
# this function eats a list and spits out the list of all the possible sublists of the input list
def powerset(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
        
    return power_set[1:]

In [14]:
# list of sublists of features
models = powerset(['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication'])
# cross validation key performance indicators
acses = np.zeros((5,len(models)))
pres = np.zeros((5,len(models)))
tprs = np.zeros((5,len(models)))
fprs = np.zeros((5,len(models)))
f1s = np.zeros((5,len(models)))
aucs = np.zeros((5,len(models)))

In [15]:
models

[['bin'],
 ['drug'],
 ['bin', 'drug'],
 ['correct_diagnosis'],
 ['bin', 'correct_diagnosis'],
 ['drug', 'correct_diagnosis'],
 ['bin', 'drug', 'correct_diagnosis'],
 ['tried_and_failed'],
 ['bin', 'tried_and_failed'],
 ['drug', 'tried_and_failed'],
 ['bin', 'drug', 'tried_and_failed'],
 ['correct_diagnosis', 'tried_and_failed'],
 ['bin', 'correct_diagnosis', 'tried_and_failed'],
 ['drug', 'correct_diagnosis', 'tried_and_failed'],
 ['bin', 'drug', 'correct_diagnosis', 'tried_and_failed'],
 ['contraindication'],
 ['bin', 'contraindication'],
 ['drug', 'contraindication'],
 ['bin', 'drug', 'contraindication'],
 ['correct_diagnosis', 'contraindication'],
 ['bin', 'correct_diagnosis', 'contraindication'],
 ['drug', 'correct_diagnosis', 'contraindication'],
 ['bin', 'drug', 'correct_diagnosis', 'contraindication'],
 ['tried_and_failed', 'contraindication'],
 ['bin', 'tried_and_failed', 'contraindication'],
 ['drug', 'tried_and_failed', 'contraindication'],
 ['bin', 'drug', 'tried_and_failed'

In [16]:
i=0
for train_index, test_index in kfold.split(X_train, y_train) : 
    X_tt = X_pre.iloc[train_index, :].copy()
    X_ho = X_pre.iloc[test_index, :].copy()
    y_tt = y_pre.iloc[train_index].copy()
    y_ho = y_pre.iloc[test_index].copy()
    # loop over all possible combinations of features
    for j in range(len(models)) :        
        if 'bin' in models[j] :
            models[j] = [feature for feature in models[j] if feature != 'bin']
            models[j].extend(['417380','417614', '417740', '999001'])
        if 'drug' in models[j] : 
            models[j] = [feature for feature in models[j] if feature != 'drug']
            models[j].extend(['A','B','C'])
            
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_tt[models[j]].values, y_tt.values)
        pred = lda.predict(X_ho[models[j]].values)
        pred_proba = lda.predict_proba(X_ho[models[j]].values)[:, 1]

        acses[i,j] = accuracy_score(y_ho.values, pred)
        conf_mat = confusion_matrix(y_ho, pred)
        pres[i,j] = precision_score(y_ho, pred)
        tprs[i,j] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
        fprs[i,j] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
        f1s[i,j] = f1_score(y_ho, pred)
        aucs[i,j] = roc_auc_score(y_ho, pred_proba)
    i=i+1

In [17]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
max_cv = np.max(cv, axis = 0)
feats.loc['LDA','accuracy_score'] = models[np.argwhere(cv == max_cv)[0][0]]

# with respect to the precision
cv = np.mean(pres, axis = 0)
max_cv = np.max(cv, axis = 0)
feats.loc['LDA','precision'] = models[np.argwhere(cv == max_cv)[0][0]]

# with respect to the true_positive_rate
cv = np.mean(tprs, axis = 0)
max_cv = np.max(cv, axis = 0)
feats.loc['LDA','true_positive_rate'] = models[np.argwhere(cv == max_cv)[0][0]]

# with respect to the false_positive_rate
cv = np.mean(fprs, axis = 0)
max_cv = np.max(cv, axis = 0)
feats.loc['LDA','false_positive_rate'] = models[np.argwhere(cv == max_cv)[0][0]]

# with respect to the f1
cv = np.mean(f1s, axis = 0)
max_cv = np.max(cv, axis = 0)
feats.loc['LDA','f1'] = models[np.argwhere(cv == max_cv)[0][0]]

# with respect to the roc_auc
cv = np.mean(aucs, axis = 0)
max_cv = np.max(cv, axis = 0)
feats.loc['LDA','roc_auc'] = models[np.argwhere(cv == max_cv)[0][0]]

In [18]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,[],[],[],[],[],[]


### -Support Vector Machine-

For support vector machine, the feature selection and hyperparameter tuning scheme is a little more complicated compared to the other models.
- First we will loosely tune hyperparameters.
- Next, using the hyperparameters previously found, we will select features by forward stepwise feature selection algorithm with respect to $R^2$ score.
- Finally, using the features previously selected, we will fine tune hyperparameters again.
- hyperparameter : margin
- we will be using `X_pre` and `y_pre` for hyperparameter tuning and feature selection.

In [19]:
from sklearn.svm import SVC

In [20]:
# hyperprameters
margins = [.1, 1, 10]
# cross validation key performance indicators
acses = np.zeros((5,len(margins)))
pres = np.zeros((5,len(margins)))
tprs = np.zeros((5,len(margins)))
fprs = np.zeros((5,len(margins)))
f1s = np.zeros((5,len(margins)))
aucs = np.zeros((5,len(margins)))

In [21]:
# loose hyperparameter tuning via cross validation
i=0
for train_index, test_index in kfold.split(X_pre, y_pre) : 
    X_tt = X_pre.iloc[train_index, :].copy()
    X_ho = X_pre.iloc[test_index, :].copy()
    y_tt = y_pre.iloc[train_index].copy()
    y_ho = y_pre.iloc[test_index].copy()
    # loop through various margins
    j=0
    for margin in margins : 
        svc = SVC(C = margin, max_iter = 10, probability = True)
        svc.fit(X_tt.values, y_tt.values)
        pred_proba = svc.decision_function(X_ho)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        
        acses[i,j] = accuracy_score(y_ho.values, 1*(pred_proba >= .5))
        conf_mat = confusion_matrix(y_ho, 1*(pred_proba >= .5))
        pres[i,j] = precision_score(y_ho, 1*(pred_proba >= .5))
        tprs[i,j] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
        fprs[i,j] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
        f1s[i,j] = f1_score(y_ho, 1*(pred_proba >= .5))
        aucs[i,j] = roc_auc_score(y_ho, pred_proba)
        j=j+1
    i=i+1

In [22]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'accuracy_score']['margin'] =  margins[margin_index]

# with respect to the precision
cv = np.mean(pres, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'precision']['margin'] =  margins[margin_index]

# with respect to the true_positive_rate
cv = np.mean(tprs, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'true_positive_rate']['margin'] =  margins[margin_index]

# with respect to the false_positive_rate
cv = np.mean(fprs, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'false_positive_rate']['margin'] =  margins[margin_index]

# with respect to the f1
cv = np.mean(f1s, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'f1']['margin'] =  margins[margin_index]

# with respect to the roc_auc
cv = np.mean(aucs, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'roc_auc']['margin'] =  margins[margin_index]

In [23]:
# forward stepwise feature selection with respect to accuracy_score

R2 = -1
added_features = []
candidate_features = ['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication']
# loop while R^2 is less than .9 and there are features that have not been added
while(R2 <= .9 and candidate_features) : 
    
    temp_features = added_features.copy()
    cv_R2 = np.zeros((len(candidate_features),5))
    
    i=0
    # loop over all the features that have not been added
    for x in candidate_features : 
        if x == 'bin' : 
            temp_features.extend(['417380','417614','417740','999001'])
        elif x == 'drug' : 
            temp_features.extend(['A','B','C'])
        else : 
            temp_features.append(x)
        j=0
        
        for train_index, test_index in kfold.split(X_pre, y_pre) :
            X_tt = X_pre[temp_features].iloc[train_index, :].copy()
            X_ho = X_pre[temp_features].iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()

            svc = SVC(C = paras.loc['SVM', 'accuracy_score']['margin'] , max_iter = 10, probability = True)
            svc.fit(X_tt.values, y_tt.values)
            pred_proba = svc.decision_function(X_ho)
            pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
            cv_R2[i,j] = r2_score(y_ho, pred_proba)
            j=j+1
            
        if x == 'bin' : 
            temp_features.remove('417380')
            temp_features.remove('417614')
            temp_features.remove('417740')
            temp_features.remove('999001')
        elif x == 'drug' : 
            temp_features.remove('A')
            temp_features.remove('B')
            temp_features.remove('C')
        else : 
            temp_features.remove(x)
        i=i+1
    
    R2 = np.max(np.mean(cv_R2, axis = 1))
    best_feature = candidate_features[np.argwhere(np.mean(cv_R2,axis=1) == R2)[0][0]]
    candidate_features.remove(best_feature)
    
    if best_feature == 'bin' : 
        added_features.extend(['417380','417614','417740','999001'])
    elif best_feature == 'drug' : 
        added_features.extend(['A','B','C'])
    else : 
        added_features.append(best_feature)


feats.loc['SVM','accuracy_score'] = added_features

In [24]:
#forward stepwise feature selection with respect to precision

R2 = -1
added_features = []
candidate_features = ['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication']
# loop while R^2 is less than .9 and there are features that have not been added
while(R2 <= .9 and candidate_features) : 
    
    temp_features = added_features.copy()
    cv_R2 = np.zeros((len(candidate_features),5))
    # loop over all the features that have not been added
    i=0
    for x in candidate_features : 
        if x == 'bin' : 
            temp_features.extend(['417380','417614','417740','999001'])
        elif x == 'drug' : 
            temp_features.extend(['A','B','C'])
        else : 
            temp_features.append(x)
        j=0
        for train_index, test_index in kfold.split(X_pre, y_pre) :
            X_tt = X_pre[temp_features].iloc[train_index, :].copy()
            X_ho = X_pre[temp_features].iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()
            
            svc = SVC(C = paras.loc['SVM', 'precision']['margin'] , max_iter = 10, probability = True)
            svc.fit(X_tt.values, y_tt.values)
            pred_proba = svc.decision_function(X_ho)
            pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
            cv_R2[i,j] = r2_score(y_ho, pred_proba)
            j=j+1
        
        if x == 'bin' : 
            temp_features.remove('417380')
            temp_features.remove('417614')
            temp_features.remove('417740')
            temp_features.remove('999001')
        elif x == 'drug' : 
            temp_features.remove('A')
            temp_features.remove('B')
            temp_features.remove('C')
        else : 
            temp_features.remove(x)
        i=i+1
    R2 = np.max(np.mean(cv_R2, axis = 1))
    best_feature = candidate_features[np.argwhere(np.mean(cv_R2,axis=1) == R2)[0][0]]
    candidate_features.remove(best_feature)
    if best_feature == 'bin' : 
        added_features.extend(['417380','417614','417740','999001'])
    elif best_feature == 'drug' : 
        added_features.extend(['A','B','C'])
    else : 
        added_features.append(best_feature)


feats.loc['SVM','precision'] = added_features

In [25]:
#forward stepwise feature selection with respect to true_positive_rate

R2 = -1
added_features = []
candidate_features = ['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication']
# loop while R^2 is less than .9 and there are features that have not been added
while(R2 <= .9 and candidate_features) : 
    
    temp_features = added_features.copy()
    cv_R2 = np.zeros((len(candidate_features),5))
    # loop over all the features that have not been added
    i=0
    for x in candidate_features : 
        if x == 'bin' : 
            temp_features.extend(['417380','417614','417740','999001'])
        elif x == 'drug' : 
            temp_features.extend(['A','B','C'])
        else : 
            temp_features.append(x)
        j=0
        for train_index, test_index in kfold.split(X_pre, y_pre) :
            X_tt = X_pre[temp_features].iloc[train_index, :].copy()
            X_ho = X_pre[temp_features].iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()
            
            svc = SVC(C = paras.loc['SVM', 'true_positive_rate']['margin'] , max_iter = 10, probability = True)
            svc.fit(X_tt.values, y_tt.values)
            pred_proba = svc.decision_function(X_ho)
            pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
            cv_R2[i,j] = r2_score(y_ho, pred_proba)
            j=j+1
        
        if x == 'bin' : 
            temp_features.remove('417380')
            temp_features.remove('417614')
            temp_features.remove('417740')
            temp_features.remove('999001')
        elif x == 'drug' : 
            temp_features.remove('A')
            temp_features.remove('B')
            temp_features.remove('C')
        else : 
            temp_features.remove(x)
        i=i+1
    R2 = np.max(np.mean(cv_R2, axis = 1))
    best_feature = candidate_features[np.argwhere(np.mean(cv_R2,axis=1) == R2)[0][0]]
    candidate_features.remove(best_feature)
    if best_feature == 'bin' : 
        added_features.extend(['417380','417614','417740','999001'])
    elif best_feature == 'drug' : 
        added_features.extend(['A','B','C'])
    else : 
        added_features.append(best_feature)


feats.loc['SVM','true_positive_rate'] = added_features

In [26]:
#forward stepwise feature selection with respect to false_positive_rate
R2 = -1
added_features = []
candidate_features = ['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication']
# loop while R^2 is less than .9 and there are features that have not been added
while(R2 <= .9 and candidate_features) : 
    
    temp_features = added_features.copy()
    cv_R2 = np.zeros((len(candidate_features),5))
    # loop over all the features that have not been added
    i=0
    for x in candidate_features : 
        if x == 'bin' : 
            temp_features.extend(['417380','417614','417740','999001'])
        elif x == 'drug' : 
            temp_features.extend(['A','B','C'])
        else : 
            temp_features.append(x)
        j=0
        for train_index, test_index in kfold.split(X_pre, y_pre) :
            X_tt = X_pre[temp_features].iloc[train_index, :].copy()
            X_ho = X_pre[temp_features].iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()
            
            svc = SVC(C = paras.loc['SVM', 'false_positive_rate']['margin'] , max_iter = 10, probability = True)
            svc.fit(X_tt.values, y_tt.values)
            pred_proba = svc.decision_function(X_ho)
            pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
            cv_R2[i,j] = r2_score(y_ho, pred_proba)
            j=j+1
         
        if x == 'bin' : 
            temp_features.remove('417380')
            temp_features.remove('417614')
            temp_features.remove('417740')
            temp_features.remove('999001')
        elif x == 'drug' : 
            temp_features.remove('A')
            temp_features.remove('B')
            temp_features.remove('C')
        else : 
            temp_features.remove(x)
        i=i+1
    R2 = np.max(np.mean(cv_R2, axis = 1))
    best_feature = candidate_features[np.argwhere(np.mean(cv_R2,axis=1) == R2)[0][0]]
    candidate_features.remove(best_feature)
    if best_feature == 'bin' : 
        added_features.extend(['417380','417614','417740','999001'])
    elif best_feature == 'drug' : 
        added_features.extend(['A','B','C'])
    else : 
        added_features.append(best_feature)


feats.loc['SVM','false_positive_rate'] = added_features

In [27]:
#forward stepwise feature selection with respect to f1
R2 = -1
added_features = []
candidate_features = ['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication']
# loop while R^2 is less than .9 and there are features that have not been added
while(R2 <= .9 and candidate_features) : 
    
    temp_features = added_features.copy()
    cv_R2 = np.zeros((len(candidate_features),5))
    # loop over all the features that have not been added
    i=0
    for x in candidate_features : 
        if x == 'bin' : 
            temp_features.extend(['417380','417614','417740','999001'])
        elif x == 'drug' : 
            temp_features.extend(['A','B','C'])
        else : 
            temp_features.append(x)
        j=0
        for train_index, test_index in kfold.split(X_pre, y_pre) :
            X_tt = X_pre[temp_features].iloc[train_index, :].copy()
            X_ho = X_pre[temp_features].iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()
            
            svc = SVC(C = paras.loc['SVM', 'f1']['margin'] , max_iter = 10, probability = True)
            svc.fit(X_tt.values, y_tt.values)
            pred_proba = svc.decision_function(X_ho)
            pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
            cv_R2[i,j] = r2_score(y_ho, pred_proba)
            j=j+1
     
        if x == 'bin' : 
            temp_features.remove('417380')
            temp_features.remove('417614')
            temp_features.remove('417740')
            temp_features.remove('999001')
        elif x == 'drug' : 
            temp_features.remove('A')
            temp_features.remove('B')
            temp_features.remove('C')
        else : 
            temp_features.remove(x)
        i=i+1
    R2 = np.max(np.mean(cv_R2, axis = 1))
    best_feature = candidate_features[np.argwhere(np.mean(cv_R2, axis = 1) == R2)[0][0]]
    candidate_features.remove(best_feature)
    if best_feature == 'bin' : 
        added_features.extend(['417380','417614','417740','999001'])
    elif best_feature == 'drug' : 
        added_features.extend(['A','B','C'])
    else : 
        added_features.append(best_feature)


feats.loc['SVM','f1'] = added_features

In [28]:
#forward stepwise feature selection with respect to roc_auc
R2 = -1
added_features = []
candidate_features = ['bin', 'drug','correct_diagnosis', 'tried_and_failed', 'contraindication']
# loop while R^2 is less than .9 and there are features that have not been added
while(R2 <= .9 and candidate_features) : 
    
    temp_features = added_features.copy()
    cv_R2 = np.zeros((len(candidate_features),5))
    # loop over all the features that have not been added
    i=0
    for x in candidate_features : 
        if x == 'bin' : 
            temp_features.extend(['417380','417614','417740','999001'])
        elif x == 'drug' : 
            temp_features.extend(['A','B','C'])
        else : 
            temp_features.append(x)
        j=0
        for train_index, test_index in kfold.split(X_pre, y_pre) :
            X_tt = X_pre[temp_features].iloc[train_index, :].copy()
            X_ho = X_pre[temp_features].iloc[test_index, :].copy()
            y_tt = y_pre.iloc[train_index].copy()
            y_ho = y_pre.iloc[test_index].copy()
            
            svc = SVC(C = paras.loc['SVM', 'roc_auc']['margin'] , max_iter = 10, probability = True)
            svc.fit(X_tt.values, y_tt.values)
            pred_proba = svc.decision_function(X_ho)
            pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
            cv_R2[i,j] = r2_score(y_ho, pred_proba)
            j=j+1
      
        if x == 'bin' : 
            temp_features.remove('417380')
            temp_features.remove('417614')
            temp_features.remove('417740')
            temp_features.remove('999001')
        elif x == 'drug' : 
            temp_features.remove('A')
            temp_features.remove('B')
            temp_features.remove('C')
        else : 
            temp_features.remove(x)
        i=i+1
    R2 = np.max(np.mean(cv_R2, axis = 1))
    best_feature = candidate_features[np.argwhere(np.mean(cv_R2,axis=1) == R2)[0][0]]
    candidate_features.remove(best_feature)
    if best_feature == 'bin' : 
        added_features.extend(['417380','417614','417740','999001'])
    elif best_feature == 'drug' : 
        added_features.extend(['A','B','C'])
    else : 
        added_features.append(best_feature)
        
feats.loc['SVM','roc_auc'] = added_features

In [29]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,"[417380, 417614, 417740, 999001, tried_and_fai...","[A, B, C, 417380, 417614, 417740, 999001, cont...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai..."


In [30]:
# final hyperparameter tuning
i=0
for train_index, test_index in kfold.split(X_pre, y_pre) : 
    X_tt = X_pre.iloc[train_index, :].copy()
    X_ho = X_pre.iloc[test_index, :].copy()
    y_tt = y_pre.iloc[train_index].copy()
    y_ho = y_pre.iloc[test_index].copy()
    # loop through various margins
    j=0
    for margin in margins : 
        svc = SVC(C = margin, max_iter = 10, probability = True)
        svc.fit(X_tt.loc[:,feats.loc['SVM','accuracy_score']].values, y_tt.values)
        pred_proba = svc.decision_function(X_ho.loc[:,feats.loc['SVM','accuracy_score']].values)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        acses[i,j] = accuracy_score(y_ho.values, 1*(pred_proba >= .5))
        
        svc.fit(X_tt.loc[:,feats.loc['SVM','precision']].values, y_tt.values)
        pred_proba = svc.decision_function(X_ho.loc[:,feats.loc['SVM','precision']].values)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        pres[i,j] = precision_score(y_ho, 1*(pred_proba >= .5))
        
        svc.fit(X_tt.loc[:,feats.loc['SVM','true_positive_rate']].values, y_tt.values)
        pred_proba = svc.decision_function(X_ho.loc[:,feats.loc['SVM','true_positive_rate']].values)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        conf_mat = confusion_matrix(y_ho, 1*(pred_proba >= .5))
        tprs[i,j] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
        
        svc.fit(X_tt.loc[:,feats.loc['SVM','false_positive_rate']].values, y_tt.values)
        pred_proba = svc.decision_function(X_ho.loc[:,feats.loc['SVM','false_positive_rate']].values)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        conf_mat = confusion_matrix(y_ho, 1*(pred_proba >= .5))
        fprs[i,j] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
        
        svc.fit(X_tt.loc[:,feats.loc['SVM','f1']].values, y_tt.values)
        pred_proba = svc.decision_function(X_ho.loc[:,feats.loc['SVM','f1']].values)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        f1s[i,j] = f1_score(y_ho, 1*(pred_proba >= .5))
        
        svc.fit(X_tt.loc[:,feats.loc['SVM','roc_auc']].values, y_tt.values)
        pred_proba = svc.decision_function(X_ho.loc[:,feats.loc['SVM','roc_auc']].values)
        pred_proba = (pred_proba - pred_proba.min()) / (pred_proba.max() - pred_proba.min())
        aucs[i,j] = roc_auc_score(y_ho, pred_proba)
        j=j+1
    i=i+1

In [31]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'accuracy_score']['margin'] =  margins[margin_index]

# with respect to the precision
cv = np.mean(pres, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'precision']['margin'] =  margins[margin_index]

# with respect to the true_positive_rate
cv = np.mean(tprs, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'true_positive_rate']['margin'] =  margins[margin_index]

# with respect to the false_positive_rate
cv = np.mean(fprs, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'false_positive_rate']['margin'] =  margins[margin_index]

# with respect to the f1
cv = np.mean(f1s, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'f1']['margin'] =  margins[margin_index]

# with respect to the roc_auc
cv = np.mean(aucs, axis = 0)
max_cv = np.max(cv, axis = 0)
margin_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['SVM', 'roc_auc']['margin'] =  margins[margin_index]

In [32]:
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,"{'penalty': 'l2', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}"
LDA,{},{},{},{},{},{}
SVM,{'margin': 10},{'margin': 0.1},{'margin': 10},{'margin': 10},{'margin': 10},{'margin': 0.1}
DT,{},{},{},{},{},{}
RF,{},{},{},{},{},{}
XGB,{},{},{},{},{},{}


In [33]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,"[417380, 417614, 417740, 999001, tried_and_fai...","[A, B, C, 417380, 417614, 417740, 999001, cont...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai..."


### -Decision Tree-

- For feature selection, we use built in pruning methods <br>
&emsp; -For pre- pruning, we use max_depth<br>
&emsp; -for post-pruning, we use cost complexity pruning
- hyperparameters : max_depth, cost_complexity constant alpha
- we will be using `X_pre` and `y_pre` for hyperparameter tuning and feature selection.

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
dt= DecisionTreeClassifier()
# hyperparameters
# cost complexity score constant
path = dt.cost_complexity_pruning_path(X_pre,y_pre)
alphas = np.append(path.ccp_alphas[:5],path.ccp_alphas[-5:])
# cross validation key performance indicators
max_depths = np.linspace(1, 32, 32, endpoint=True)
acses = np.zeros((5,len(max_depths), len(alphas)))
pres = np.zeros((5,len(max_depths), len(alphas)))
tprs = np.zeros((5,len(max_depths), len(alphas)))
fprs = np.zeros((5,len(max_depths), len(alphas)))
f1s = np.zeros((5,len(max_depths), len(alphas)))
aucs = np.zeros((5,len(max_depths), len(alphas)))

In [36]:
i=0
for train_index, test_index in kfold.split(X_pre, y_pre) : 
    # data split into training set and holdout set
    X_tt = X_pre.iloc[train_index, :].copy()
    X_ho = X_pre.iloc[test_index, :].copy()
    y_tt = y_pre.iloc[train_index].copy()
    y_ho = y_pre.iloc[test_index].copy()
    j=0
    # loop through various max_depths 
    for max_depth in max_depths :
        k=0
        # loop through various cost complexity score constants
        for ccp_alpha in alphas :
            # fit the model against the training data and transform the holdout data
            dt = DecisionTreeClassifier(max_depth = max_depth)
            dt.fit(X_tt.values, y_tt.values)
            pred = dt.predict(X_ho.values)
            pred_proba = dt.predict_proba(X_ho.values)[:,1]
            #store various scores for each fold of the cross validation
            acses[i,j,k] = accuracy_score(y_ho.values, pred)
            conf_mat = confusion_matrix(y_ho, pred)
            pres[i,j,k] = precision_score(y_ho, pred)
            tprs[i,j,k] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
            fprs[i,j,k] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
            f1s[i,j,k] = f1_score(y_ho, pred)
            aucs[i,j,k] = roc_auc_score(y_ho, pred_proba)
            k=k+1
        j=j+1
    i=i+1

In [37]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
alpha_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['DT', 'accuracy_score']['max_depth'] = max_depths[depth_index]
paras.loc['DT', 'accuracy_score']['ccp_alpha'] = alphas[alpha_index]

# with respect to the precision
cv = np.mean(pres, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
alpha_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['DT', 'precision']['max_depth'] = max_depths[depth_index]
paras.loc['DT', 'precision']['ccp_alpha'] = alphas[alpha_index]

# with respect to the true_positive_rate
cv = np.mean(tprs, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
alpha_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['DT', 'true_positive_rate']['max_depth'] = max_depths[depth_index]
paras.loc['DT', 'true_positive_rate']['ccp_alpha'] = alphas[alpha_index]

# with respect to the false_positive_rate
cv = np.mean(fprs, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
alpha_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['DT', 'false_positive_rate']['max_depth'] = max_depths[depth_index]
paras.loc['DT', 'false_positive_rate']['ccp_alpha'] = alphas[alpha_index]

# with respect to the f1
cv = np.mean(f1s, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
alpha_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['DT', 'f1']['max_depth'] = max_depths[depth_index]
paras.loc['DT', 'f1']['ccp_alpha'] = alphas[alpha_index]

# with respect to the roc_auc
cv = np.mean(aucs, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
alpha_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['DT', 'roc_auc']['max_depth'] = max_depths[depth_index]
paras.loc['DT', 'roc_auc']['ccp_alpha'] = alphas[alpha_index]

In [38]:
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,"{'penalty': 'l2', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}"
LDA,{},{},{},{},{},{}
SVM,{'margin': 10},{'margin': 0.1},{'margin': 10},{'margin': 10},{'margin': 10},{'margin': 0.1}
DT,"{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}"
RF,{},{},{},{},{},{}
XGB,{},{},{},{},{},{}


In [39]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,"[417380, 417614, 417740, 999001, tried_and_fai...","[A, B, C, 417380, 417614, 417740, 999001, cont...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai..."


### -Random Forest-

- For feature selection we use built in pruning method :<br>
\-For pre- pruning, we use max_depth.<br> 
\-Unlike decision tree model,we do not perform post_pruning using cost_complexity_score because post_pruning usually sub-optimize the performance of ensemble methods
- hyperparamete : max_depth
- We will be using `X_pre` and `y_pre` for hyperparameter tuning and feature selection.

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
# hyperparameters
max_depths = [3 ,4, 5, 6]
# cross validation key performance indicators
acses = np.zeros((5,len(max_depths)))
pres = np.zeros((5,len(max_depths)))
tprs = np.zeros((5,len(max_depths)))
fprs = np.zeros((5,len(max_depths)))
f1s = np.zeros((5,len(max_depths)))
aucs = np.zeros((5,len(max_depths)))

In [42]:
# hyperparameter tuning via cross validation
i=0
for train_index, test_index in kfold.split(X_pre, y_pre) :
    X_tt = X_pre.iloc[train_index, :].copy()
    X_ho = X_pre.iloc[test_index, :].copy()
    y_tt = y_pre.iloc[train_index].copy()
    y_ho = y_pre.iloc[test_index].copy()
    j=0
    # loop through various max_depths
    for max_depth in max_depths :
        rf = RandomForestClassifier(n_estimators = 100,
                                    max_depth = max_depth,
                                    random_state = 623)
        rf.fit(X_tt.values,y_tt.values)
        pred = rf.predict(X_ho.values)
        pred_proba = rf.predict_proba(X_ho.values)[:,1]
        
        acses[i,j] = accuracy_score(y_ho.values, pred)
        conf_mat = confusion_matrix(y_ho, pred)
        pres[i,j] = precision_score(y_ho, pred)
        tprs[i,j] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
        fprs[i,j] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
        f1s[i,j] = f1_score(y_ho.values, pred)
        aucs[i,j] = roc_auc_score(y_ho.values, pred_proba)
        j=j+1
    i=i+1        

In [43]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['RF', 'accuracy_score']['max_depth'] = max_depths[depth_index]

# with respect to the precision
cv = np.mean(pres, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['RF', 'precision']['max_depth'] = max_depths[depth_index]

# with respect to the true_positive_rate
cv = np.mean(tprs, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['RF', 'true_positive_rate']['max_depth'] = max_depths[depth_index]

# with respect to the false_positive_rate
cv = np.mean(fprs, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['RF', 'false_positive_rate']['max_depth'] = max_depths[depth_index]

# with respect to the f1
cv = np.mean(f1s, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['RF', 'f1']['max_depth'] = max_depths[depth_index]

# with respect to the roc_auc
cv = np.mean(aucs, axis = 0)
max_cv = np.max(cv, axis = 0)
depth_index = np.argwhere(cv == max_cv)[0][0]
paras.loc['RF', 'roc_auc']['max_depth'] = max_depths[depth_index]

In [44]:
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,"{'penalty': 'l2', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}"
LDA,{},{},{},{},{},{}
SVM,{'margin': 10},{'margin': 0.1},{'margin': 10},{'margin': 10},{'margin': 10},{'margin': 0.1}
DT,"{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}"
RF,{'max_depth': 6},{'max_depth': 6},{'max_depth': 3},{'max_depth': 3},{'max_depth': 6},{'max_depth': 6}
XGB,{},{},{},{},{},{}


In [45]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,"[417380, 417614, 417740, 999001, tried_and_fai...","[A, B, C, 417380, 417614, 417740, 999001, cont...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai..."


### -XGBoost-

- For feature selection, we use built in pre-pruning method by fixing max_depth of individual tree.
- hyperparameter tuning : learning rates, max_depth
- We will be using `X_pre` and `y_pre` for hyperparameter tuning and feature selection.

In [46]:
from xgboost import XGBClassifier

In [47]:
# hyperparameters
learning_rates = [0.01, 0.1, 1, 10, 100]
max_depths = [3, 5, 10]
# cross validation key performance indicators
acses = np.zeros((5,len(learning_rates), len(max_depths)))
pres = np.zeros((5,len(learning_rates), len(max_depths)))
tprs = np.zeros((5,len(learning_rates), len(max_depths)))
fprs = np.zeros((5,len(learning_rates), len(max_depths)))
f1s = np.zeros((5,len(learning_rates), len(max_depths)))
aucs = np.zeros((5,len(learning_rates), len(max_depths)))

In [48]:
# hyperparameter tuning via cross validation
i=0
for train_index, test_index in kfold.split(X_pre, y_pre) : 
    X_tt = X_pre.iloc[train_index, :].copy()
    X_ho = X_pre.iloc[test_index, :].copy()
    y_tt = y_pre.iloc[train_index].copy()
    y_ho = y_pre.iloc[test_index].copy()
    # loop through various learning rates
    j=0
    for learning_rate in learning_rates :
        # loop through various max_depths
        k=0
        for max_depth in max_depths :
            xgb = XGBClassifier(learning_rate = learning_rate,
                                    max_depth = max_depth,
                                    use_label_encoder=False,
                                    n_estimators = 100,
                                    random_state = 623,
                                    eval_metric = 'logloss')
            xgb.fit(X_tt.values, y_tt.values,
                    early_stopping_rounds = 3,
                    eval_set = [(X_ho.values, y_ho.values)])
            pred = xgb.predict(X_ho.values)
            pred_proba = xgb.predict_proba(X_ho.values)[:,1]
            
            acses[i,j,k] = accuracy_score(y_ho.values, pred)
            conf_mat = confusion_matrix(y_ho, pred)
            pres[i,j,k] = precision_score(y_ho, pred)
            tprs[i,j,k] = conf_mat[1,1]/(conf_mat[1,0] + conf_mat[1,1])
            fprs[i,j,k] = conf_mat[0,1]/(conf_mat[0,0] + conf_mat[0,1])
            f1s[i,j,k] = f1_score(y_ho, pred)
            aucs[i,j,k] = roc_auc_score(y_ho, pred_proba)
            k=k+1
        j=j+1
    i=i+1        

[0]	validation_0-logloss:0.68990
[1]	validation_0-logloss:0.68680
[2]	validation_0-logloss:0.68372
[3]	validation_0-logloss:0.68071
[4]	validation_0-logloss:0.67771
[5]	validation_0-logloss:0.67477
[6]	validation_0-logloss:0.67193
[7]	validation_0-logloss:0.66914
[8]	validation_0-logloss:0.66637
[9]	validation_0-logloss:0.66372
[10]	validation_0-logloss:0.66108
[11]	validation_0-logloss:0.65845
[12]	validation_0-logloss:0.65594
[13]	validation_0-logloss:0.65343
[14]	validation_0-logloss:0.65100
[15]	validation_0-logloss:0.64857
[16]	validation_0-logloss:0.64623
[17]	validation_0-logloss:0.64391
[18]	validation_0-logloss:0.64166
[19]	validation_0-logloss:0.63941
[20]	validation_0-logloss:0.63723
[21]	validation_0-logloss:0.63507
[22]	validation_0-logloss:0.63293
[23]	validation_0-logloss:0.63086
[24]	validation_0-logloss:0.62883
[25]	validation_0-logloss:0.62681
[26]	validation_0-logloss:0.62483
[27]	validation_0-logloss:0.62292
[28]	validation_0-logloss:0.62097
[29]	validation_0-loglos

In [49]:
# tuned hyperparameters

# with respect to the accuracy_score
cv = np.mean(acses, axis = 0)
max_cv = np.max(np.max(cv, axis = 0),axis = 0)
rate_index = np.argwhere(cv == max_cv)[0][0]
depth_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['XGB', 'accuracy_score']['learning_rate'] = learning_rates[rate_index]
paras.loc['XGB', 'accuracy_score']['max_depth'] = max_depths[depth_index]

# with respect to the precision
cv = np.mean(pres, axis = 0)
max_cv = np.max(np.max(cv, axis = 0),axis = 0)
rate_index = np.argwhere(cv == max_cv)[0][0]
depth_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['XGB', 'precision']['learning_rate'] = learning_rates[rate_index]
paras.loc['XGB', 'precision']['max_depth'] = max_depths[depth_index]

# with respect to the true_positive_rate
cv = np.mean(tprs, axis = 0)
max_cv = np.max(np.max(cv, axis = 0),axis = 0)
rate_index = np.argwhere(cv == max_cv)[0][0]
depth_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['XGB', 'true_positive_rate']['learning_rate'] = learning_rates[rate_index]
paras.loc['XGB', 'true_positive_rate']['max_depth'] = max_depths[depth_index]

# with respect to the false_positive_rate
cv = np.mean(fprs, axis = 0)
max_cv = np.max(np.max(cv, axis = 0),axis = 0)
rate_index = np.argwhere(cv == max_cv)[0][0]
depth_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['XGB', 'false_positive_rate']['learning_rate'] = learning_rates[rate_index]
paras.loc['XGB', 'false_positive_rate']['max_depth'] = max_depths[depth_index]

# with respect to the f1
cv = np.mean(f1s, axis = 0)
max_cv = np.max(np.max(cv, axis = 0),axis = 0)
rate_index = np.argwhere(cv == max_cv)[0][0]
depth_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['XGB', 'f1']['learning_rate'] = learning_rates[rate_index]
paras.loc['XGB', 'f1']['max_depth'] = max_depths[depth_index]

# with respect to the roc_auc
cv = np.mean(aucs, axis = 0)
max_cv = np.max(np.max(cv, axis = 0),axis = 0)
rate_index = np.argwhere(cv == max_cv)[0][0]
depth_index = np.argwhere(cv == max_cv)[0][1]
paras.loc['XGB', 'roc_auc']['learning_rate'] = learning_rates[rate_index]
paras.loc['XGB', 'roc_auc']['max_depth'] = max_depths[depth_index]

In [50]:
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,"{'penalty': 'l2', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}"
LDA,{},{},{},{},{},{}
SVM,{'margin': 10},{'margin': 0.1},{'margin': 10},{'margin': 10},{'margin': 10},{'margin': 0.1}
DT,"{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}"
RF,{'max_depth': 6},{'max_depth': 6},{'max_depth': 3},{'max_depth': 3},{'max_depth': 6},{'max_depth': 6}
XGB,"{'learning_rate': 1, 'max_depth': 3}","{'learning_rate': 0.01, 'max_depth': 10}","{'learning_rate': 0.01, 'max_depth': 3}","{'learning_rate': 0.01, 'max_depth': 3}","{'learning_rate': 0.1, 'max_depth': 3}","{'learning_rate': 1, 'max_depth': 10}"


In [51]:
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,"[417380, 417614, 417740, 999001, tried_and_fai...","[A, B, C, 417380, 417614, 417740, 999001, cont...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai..."


### -Tuned Hyperparameters and Selected Features-

In [52]:
#tuned hyperparameters
paras

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
Base,{},{},{},{},{},{}
Log,"{'penalty': 'l2', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}","{'penalty': 'elasticnet', 'C': 0.01}"
LDA,{},{},{},{},{},{}
SVM,{'margin': 10},{'margin': 0.1},{'margin': 10},{'margin': 10},{'margin': 10},{'margin': 0.1}
DT,"{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 7.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 1.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}","{'max_depth': 8.0, 'ccp_alpha': 0.0}"
RF,{'max_depth': 6},{'max_depth': 6},{'max_depth': 3},{'max_depth': 3},{'max_depth': 6},{'max_depth': 6}
XGB,"{'learning_rate': 1, 'max_depth': 3}","{'learning_rate': 0.01, 'max_depth': 10}","{'learning_rate': 0.01, 'max_depth': 3}","{'learning_rate': 0.01, 'max_depth': 3}","{'learning_rate': 0.1, 'max_depth': 3}","{'learning_rate': 1, 'max_depth': 10}"


In [53]:
# selected features
feats

,accuracy_score,precision,true_positive_rate,false_positive_rate,f1,roc_auc
LDA,"[contraindication, 417380, 417614, 417740, 999...","[contraindication, 417380, 417614, 417740, 999...","[417380, 417614, 417740, 999001]","[417380, 417614, 417740, 999001]","[contraindication, 417380, 417614, 417740, 999...","[correct_diagnosis, tried_and_failed, contrain..."
SVM,"[417380, 417614, 417740, 999001, tried_and_fai...","[A, B, C, 417380, 417614, 417740, 999001, cont...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai...","[417380, 417614, 417740, 999001, tried_and_fai..."


# 2. Save Selected Features and Tuned Hyperparameters

In [54]:
# save dataframes as csv files
paras.to_csv('data/paras',header = True, index = True)
feats.to_csv('data/feats',header = True, index = True)